In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

## importing the data from the mobile operator in zamalek area

In [3]:
towerloc = pd.read_csv("towerdata.csv") 

In [4]:
towerloc

,eNodeB,traffic,lat,lon
0,LCAIW20945,150,30.069542,31.218173
1,LCAIW20459,50,30.066012,31.221055
2,LCAIW20298,70,30.062500,31.217450
3,LCAIW20461,100,30.061819,31.223215
4,LCAIW20293,30,30.056056,31.224817
5,LCAIW20300,80,30.066537,31.230656
6,LCAIW20944,40,30.058266,31.229535


## creating geo polygons in the area to relate the sites tower and venues in them

In [5]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=36, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=36, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
#print('Zamalek center longitude={}, latitude={}'.format(berlin_center[1], berlin_center[0]))
x, y = lonlat_to_xy(31.220228, 30.063513)
print('zamalek center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Zamalek center longitude={}, latitude={}'.format(lo, la))

     |████████████████████████████████| 1.5MB 24.0MB/s eta 0:00:01
Coordinate transformation check
-------------------------------
zamalek center UTM X=328440.67704183457, Y=3327158.2514471216
Zamalek center longitude=31.220227999999995, latitude=30.063512999999997


In [6]:
zamalek_center_x, zamalek_center_y = lonlat_to_xy(31.220228, 30.063513) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = zamalek_center_x - 6000
x_step = 600
y_min = zamalek_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(zamalek_center_x, zamalek_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


In [7]:
zamalek_center = (31.220228, 30.063513)
map_zamalek = folium.Map(location=zamalek_center, zoom_start=13)
folium.Marker(zamalek_center, popup='Alexanderplatz').add_to(map_zamalek)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_zamalek)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_zamalek

## conneting to foursqure to get the venurs data in zamalrk area

In [8]:
CLIENT_ID = '43X1WLFVVAOIBBJWWT2MP4WHPHARAGBEUVR3DKJ1TS4ZS35F' # your Foursquare ID
CLIENT_SECRET = 'M30L3ASI2XPQRDGY31WPKYAKYSZP3VVRAKINQWKFSGLX4J2U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 43X1WLFVVAOIBBJWWT2MP4WHPHARAGBEUVR3DKJ1TS4ZS35F
CLIENT_SECRET:M30L3ASI2XPQRDGY31WPKYAKYSZP3VVRAKINQWKFSGLX4J2U


In [9]:
neighborhood_latitude = 30.063513 # neighborhood latitude value
neighborhood_longitude = 31.220228 # neighborhood longitude value

In [10]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API



radius = 1000 # define radius




url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=43X1WLFVVAOIBBJWWT2MP4WHPHARAGBEUVR3DKJ1TS4ZS35F&client_secret=M30L3ASI2XPQRDGY31WPKYAKYSZP3VVRAKINQWKFSGLX4J2U&v=20180605&ll=30.063513,31.220228&radius=1000&limit=1000'

In [11]:
results = requests.get(url).json()

In [12]:
results

{'meta': {'code': 200, 'requestId': '5dbd71742b274a0039842dde'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Zamalek',
  'headerFullLocation': 'Zamalek, Cairo',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 134,
  'suggestedBounds': {'ne': {'lat': 30.07251300900001,
    'lng': 31.23060756088153},
   'sw': {'lat': 30.054512990999992, 'lng': 31.209848439118467}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57d1b278498e4bb2907a76e9',
       'name': 'Zamalek Cinema',
       'location': {'address': '13 Shagaret El Durr St',
        'crossStreet': '26 of July St',
        'lat': 30.061760222603002,
        'lng': 31.218793667827992,
        'labeledLatLngs': [{

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Zamalek Cinema,Multiplex,30.061760,31.218794
1,Zööba (زووبا),Middle Eastern Restaurant,30.061248,31.219263
2,Maison 69,Boutique,30.063842,31.218536
3,Mandarine Koueider,Pastry Shop,30.062634,31.219732
4,Villa Baboushka,Boutique,30.062980,31.221455


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [16]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 51 uniques categories.


In [17]:
# one hot encoding
onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = 'Zamalek'

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bar,Boat or Ferry,Bookstore,Boutique,Bubble Tea Shop,Burger Joint,Café,Coffee Shop,Cupcake Shop,Dessert Shop,Eastern European Restaurant,Food Stand,Gym,Gym / Fitness Center,Health & Beauty Service,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lebanese Restaurant,Lounge,Middle Eastern Restaurant,Motorcycle Shop,Multiplex,Nightclub,Other Nightlife,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pie Shop,Pizza Place,Pub,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Waterfront,Yoga Studio
0,Zamalek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Zamalek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Zamalek,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Zamalek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Zamalek,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = onehot['Neighborhood']

for ind in np.arange(onehot.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(onehot.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Zamalek,Multiplex,Yoga Studio,Jazz Club,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hotel Bar,Hotel,Health & Beauty Service,Gym / Fitness Center
1,Zamalek,Middle Eastern Restaurant,Yoga Studio,Jazz Club,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hotel Bar,Hotel,Health & Beauty Service,Gym / Fitness Center
2,Zamalek,Boutique,Yoga Studio,Dessert Shop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hotel Bar,Hotel,Health & Beauty Service,Gym / Fitness Center
3,Zamalek,Pastry Shop,Yoga Studio,Jazz Club,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hotel Bar,Hotel,Health & Beauty Service,Gym / Fitness Center
4,Zamalek,Boutique,Yoga Studio,Dessert Shop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hotel Bar,Hotel,Health & Beauty Service,Gym / Fitness Center


In [20]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = onehot.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [21]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = nearby_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = onehot.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bar,Boat or Ferry,Bookstore,Boutique,Bubble Tea Shop,Burger Joint,Café,Coffee Shop,Cupcake Shop,Dessert Shop,Eastern European Restaurant,Food Stand,Gym,Gym / Fitness Center,Health & Beauty Service,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lebanese Restaurant,Lounge,Middle Eastern Restaurant,Motorcycle Shop,Multiplex,Nightclub,Other Nightlife,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pie Shop,Pizza Place,Pub,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Waterfront,Yoga Studio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Zamalek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Multiplex,Yoga Studio,Jazz Club,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hotel Bar,Hotel,Health & Beauty Service,Gym / Fitness Center
0,Zamalek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Middle Eastern Restaurant,Yoga Studio,Jazz Club,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hotel Bar,Hotel,Health & Beauty Service,Gym / Fitness Center
0,Zamalek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Boutique,Yoga Studio,Dessert Shop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hotel Bar,Hotel,Health & Beauty Service,Gym / Fitness Center
0,Zamalek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pastry Shop,Yoga Studio,Jazz Club,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hotel Bar,Hotel,Health & Beauty Service,Gym / Fitness Center
0,Zamalek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Boutique,Yoga Studio,Dessert Shop,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Hotel Bar,Hotel,Health & Beauty Service,Gym / Fitness Center


## getting the nearest serving site or tower to the foursquare venue

In [22]:
import pandas as pd
from scipy.spatial.distance import cdist

In [23]:
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

In [24]:

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

In [25]:

df1 = pd.DataFrame(towerloc)
df2 = pd.DataFrame(nearby_venues)

In [26]:
df2

,name,categories,lat,lng
0,Zamalek Cinema,Multiplex,30.061760,31.218794
1,Zööba (زووبا),Middle Eastern Restaurant,30.061248,31.219263
2,Maison 69,Boutique,30.063842,31.218536
3,Mandarine Koueider,Pastry Shop,30.062634,31.219732
4,Villa Baboushka,Boutique,30.062980,31.221455
5,Mince,Burger Joint,30.065544,31.221254
6,NOLA Cupcakes,Cupcake Shop,30.061364,31.222698
7,Azza Fahmy,Jewelry Store,30.065929,31.220257
8,ABnG Gallery,Art Gallery,30.063805,31.219896
9,Bubblicious,Bubble Tea Shop,30.065156,31.217279


In [27]:

df1['point'] = [(x, y) for x,y in zip(df1['lat'], df1['lon'])]
df2['point'] = [(x, y) for x,y in zip(df2['lat'], df2['lng'])]

In [28]:
df2['closest'] = [closest_point(x, list(df1['point'])) for x in df2['point']]


In [29]:
df2

,name,categories,lat,lng,point,closest
0,Zamalek Cinema,Multiplex,30.061760,31.218794,"(30.061760222603002, 31.218793667827992)","(30.0625, 31.217449900000002)"
1,Zööba (زووبا),Middle Eastern Restaurant,30.061248,31.219263,"(30.06124837014216, 31.219262645315787)","(30.0625, 31.217449900000002)"
2,Maison 69,Boutique,30.063842,31.218536,"(30.063842, 31.218536)","(30.0625, 31.217449900000002)"
3,Mandarine Koueider,Pastry Shop,30.062634,31.219732,"(30.06263394562906, 31.219732275388324)","(30.0625, 31.217449900000002)"
4,Villa Baboushka,Boutique,30.062980,31.221455,"(30.06298, 31.221455)","(30.061819, 31.223215000000003)"
5,Mince,Burger Joint,30.065544,31.221254,"(30.06554413385268, 31.22125433658391)","(30.066012, 31.221055)"
6,NOLA Cupcakes,Cupcake Shop,30.061364,31.222698,"(30.061363616463666, 31.222697594783313)","(30.061819, 31.223215000000003)"
7,Azza Fahmy,Jewelry Store,30.065929,31.220257,"(30.065928600364355, 31.220256879003998)","(30.066012, 31.221055)"
8,ABnG Gallery,Art Gallery,30.063805,31.219896,"(30.063805068319862, 31.2198962898472)","(30.066012, 31.221055)"
9,Bubblicious,Bubble Tea Shop,30.065156,31.217279,"(30.065155534332987, 31.217278507457564)","(30.0625, 31.217449900000002)"


In [34]:
df2['zone'] = [match_value(df1, 'point', x, 'eNodeB') for x in df2['closest']]

In [35]:
df2

,name,categories,lat,lng,point,closest,zone
0,Zamalek Cinema,Multiplex,30.061760,31.218794,"(30.061760222603002, 31.218793667827992)","(30.0625, 31.217449900000002)",LCAIW20298
1,Zööba (زووبا),Middle Eastern Restaurant,30.061248,31.219263,"(30.06124837014216, 31.219262645315787)","(30.0625, 31.217449900000002)",LCAIW20298
2,Maison 69,Boutique,30.063842,31.218536,"(30.063842, 31.218536)","(30.0625, 31.217449900000002)",LCAIW20298
3,Mandarine Koueider,Pastry Shop,30.062634,31.219732,"(30.06263394562906, 31.219732275388324)","(30.0625, 31.217449900000002)",LCAIW20298
4,Villa Baboushka,Boutique,30.062980,31.221455,"(30.06298, 31.221455)","(30.061819, 31.223215000000003)",LCAIW20461
5,Mince,Burger Joint,30.065544,31.221254,"(30.06554413385268, 31.22125433658391)","(30.066012, 31.221055)",LCAIW20459
6,NOLA Cupcakes,Cupcake Shop,30.061364,31.222698,"(30.061363616463666, 31.222697594783313)","(30.061819, 31.223215000000003)",LCAIW20461
7,Azza Fahmy,Jewelry Store,30.065929,31.220257,"(30.065928600364355, 31.220256879003998)","(30.066012, 31.221055)",LCAIW20459
8,ABnG Gallery,Art Gallery,30.063805,31.219896,"(30.063805068319862, 31.2198962898472)","(30.066012, 31.221055)",LCAIW20459
9,Bubblicious,Bubble Tea Shop,30.065156,31.217279,"(30.065155534332987, 31.217278507457564)","(30.0625, 31.217449900000002)",LCAIW20298


## pivot table to get count of venues near to the tower or site

In [36]:
zamalekall=df2

In [37]:
pivot = zamalekall.pivot_table(index=['zone'], aggfunc='count')


In [38]:
print(pivot)

            categories  closest  lat  lng  name  point
zone                                                  
LCAIW20293          20       20   20   20    20     20
LCAIW20298          26       26   26   26    26     26
LCAIW20300           8        8    8    8     8      8
LCAIW20459          14       14   14   14    14     14
LCAIW20461          22       22   22   22    22     22
LCAIW20945          10       10   10   10    10     10
